# Demo of a Simple Calculator Agent

In this notebook, we'll see how to write and use a very simple Calculator agent. It will support one operation: adding numbers based on text commands provided by the user. 

Let's start with various imports we need.


In [6]:
# Make a file `.env` and put your OPEN_AI_KEY there.

import os
from dotenv import load_dotenv
load_dotenv()
OPEN_AI_KEY = os.environ.get('OPEN_AI_KEY')

In [7]:
from toolfuse import Tool, action
from rich import print
from threadmem import RoleThread
from litellm import completion
from skillpacks.server.models import V1ActionSelection
import json
import logging
from tenacity import before_sleep_log, retry, stop_after_attempt

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


Now, let's define the Calculator Tool. It's a simple implementation of the Tool class, with one method: add.

Note that even though this calculator has only one method, typically a tool would have one more actions, potentially very complex ones.

In [11]:
class Calculator(Tool):
    def __init__(self):
        super().__init__()

    @action
    def add(self, a: int, b: int) -> int:
        return a + b

Now, let's define the Thinker class which manages the interaction between the LLM and the tool. It performs the following key functions:

- Initializes the `Calculator` tool and a `RoleThread` for message threading
- Retrieves the JSON schema of the tool, which outlines its available actions and parameters.
- Crafts a system prompt that instructs the LLM on how to utilize the tool and format responses
- The `ask` method in the `Thinker` class accepts and processes user request.
- The `ask` method calls the `request` method to call the LLM, parse its response and then execute the action to get the final result for the user.

In [9]:
class Thinker():
    def __init__(self):
        self.tool = Calculator()
        self.thread = RoleThread()

        # Then, we can see the JSON schema of the tool: 
        schema = self.tool.json_schema()
        print(f"Tool schema:\n{schema}")

        # Now, to use the tool with an LLM, we need to give an LLM a few things:
        # 1. The tool schema -- the list of tools an LLM can use
        # 2. The response JSON schema -- the format of the response an LLM should give us
        # 3. The task -- the specific task we want the LLM to perform

        system_prompt = f"""
        You are a helpful assistant able to perform complex mathematical operations. 
        You have access to the following tools: {schema}. 
        For each task that I send to you, you have to return your thoughts and the specific tool to use to solve it.
        Please return your response in the following JSON format: {V1ActionSelection.model_json_schema()}
        Please only return the JSON object without any additional text or formatting.
        """
        self.thread.post("system", system_prompt)

    def ask(self, question: str) -> str:
        thread_copy = self.thread.copy()
        thread_copy, result = self.request(thread_copy, question)
        self.thread = thread_copy
        return result
    
    # We wrap the actual request in a retry decorator to handle incorrect response format. 
    # If we receive a response that is not in the correct format, the Exception will be raised and the request will be retried.
    @retry(
        stop=stop_after_attempt(5),
        before_sleep=before_sleep_log(logger, logging.INFO),            
    )
    def request(self, thread: RoleThread, question: str) -> str:
        thread.post("user", question)

        response = completion(model="gpt-4o", messages=thread.to_openai())
        print(f"Raw response:\n{response}")

        # At this point, we expect the response to be in the format of V1ActionSelection.json_schema()
        # So we can just parse the response as a V1ActionSelection object

        action_selection = V1ActionSelection(**json.loads(response.choices[0].message.content))
        print(f"Suggested action:\n{action_selection}")

        # Now, when we have the action selection in JSON format, we use the tool to find and run this action

        action_object = self.tool.find_action(action_selection.action.name)
        print(f"Action object:\n{action_object}")

        result = self.tool.use(action_object, **action_selection.action.parameters)
        print(f"Action result:\n{result}")

        return thread, result

Now, you can use the tool.

First, create an instance of the `Thinker` class.


In [15]:
thinker = Thinker()

Tool schema:
[{'name': 'add', 'parameters': {'type': 'object', 'properties': {'a': {'type': 'integer'}, 'b': {'type': 
'integer'}}, 'required': ['a', 'b']}}]

Now, call the `ask` method with the user request.

The `ask` method passes the tool schema to the LLM. The LLM parses the user request and returns the name of the method (`ad`) and the two numbers to be added.

Then the method is executed to provide the final result to the user.

In [16]:
result_1 = thinker.ask("What is 1234 + 8765?")
print(f"Result 1:\n{result_1}")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
16:49:06 - LiteLLM:INFO: utils.py:3401 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler


Raw response:
ModelResponse(id='chatcmpl-AJIaSsIGwEUPoL0STGN7NAJqkesp6', choices=[Choices(finish_reason='stop', index=0, 
message=Message(content='{\n    "observation": "The task is to find the sum of two integers, 1234 and 8765.",\n    
"reason": "The appropriate action to take is to use the add tool, which is specifically designed to perform 
addition of two integers.",\n    "action": {\n        "name": "add",\n        "parameters": {\n            "a": 
1234,\n            "b": 8765\n        }\n    },\n    "expectation": "The sum of 1234 and 8765 should be 
returned."\n}', role='assistant'))], created=1729163044, model='gpt-4o-2024-08-06', object='chat.completion', 
system_fingerprint='fp_e5e4913e83', usage=Usage(completion_tokens=112, prompt_tokens=445, total_tokens=557))

Suggested action:
observation='The task is to find the sum of two integers, 1234 and 8765.' reason='The appropriate action to take is
to use the add tool, which is specifically designed to perform addition of two integers.' 
action=V1Action(name='add', parameters={'a': 1234, 'b': 8765}) expectation='The sum of 1234 and 8765 should be 
returned.'

Action object:
<toolfuse.base.Action object at 0x13154e190>

Action result:
9999

Result 1:
9999

The LLM can also parse the user request when the input numbers provided as words.

In [17]:
result_2 = thinker.ask("How much do I get if I add two and seven?")
print(f"Result 2:\n{result_2}")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
16:49:08 - LiteLLM:INFO: utils.py:3401 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler


Raw response:
ModelResponse(id='chatcmpl-AJIaVuCpU1A7MPqms1PgKBRii6Hmq', choices=[Choices(finish_reason='stop', index=0, 
message=Message(content='{"observation":"The question is asking for the sum of 2 and 7.","reason":"To find the sum,
I will use the addition tool with the given numbers 2 and 7 as 
inputs.","action":{"name":"add","parameters":{"a":2,"b":7}},"expectation":"The sum of 2 and 7, which is 9, should 
be returned."}', role='assistant'))], created=1729163047, model='gpt-4o-2024-08-06', object='chat.completion', 
system_fingerprint='fp_e5e4913e83', usage=Usage(completion_tokens=82, prompt_tokens=461, total_tokens=543))

Suggested action:
observation='The question is asking for the sum of 2 and 7.' reason='To find the sum, I will use the addition tool 
with the given numbers 2 and 7 as inputs.' action=V1Action(name='add', parameters={'a': 2, 'b': 7}) 
expectation='The sum of 2 and 7, which is 9, should be returned.'

Action object:
<toolfuse.base.Action object at 0x13154e190>

Action result:
9

Result 2:
9